# Kapitel 4: Eleganz und Effizienz – Gekoppelte Daten & neue Widgets

In den letzten Kapiteln haben wir unsere Anwendung zum Leben erweckt, aber auf eine recht umständliche Weise. Wir mussten explizit einen Button klicken, um Daten mit `.get()` zu holen und die Oberfläche mit `.config()` zu aktualisieren. In diesem Kapitel lernen wir eine weitaus elegantere Methode kennen: die direkte Datenbindung. Wir lassen unsere Widgets direkt mit den Daten unseres Programms "sprechen".

## Die Theorie: Von manueller Arbeit zur magischen Synchronisation

### 4.1 Das Problem: Manuelle Synchronisation

Stellen wir uns den bisherigen Datenfluss vor, um einen Text aus einem Eingabefeld in einem Label anzuzeigen:

1.  Benutzer tippt Text in das `Entry`.
2.  Benutzer klickt auf einen `Button`.
3.  Der `command` des Buttons ruft eine `callback`-Funktion auf.
4.  Die Funktion holt den Text mit `entry.get()`.
5.  Die Funktion aktualisiert das Label mit `label.config(text=...)`.

Das sind fünf Schritte für eine simple Aufgabe. Es ist nicht "live" und erfordert immer eine explizite Aktion des Benutzers. Das ist umständlich und für viele Anwendungsfälle nicht ideal.

### 4.2 Die Lösung: Tkinter-Kontrollvariablen

Tkinter bietet eine brillante Lösung für dieses Problem: **Kontrollvariablen**. Man kann sich diese Variablen wie ein "gemeinsames Gehirn" für mehrere Widgets vorstellen.

  * **Was sie sind:** Spezielle Tkinter-Objekte (nicht zu verwechseln mit normalen Python-Variablen), die dazu dienen, Daten zu speichern, die von Widgets gemeinsam genutzt werden.
  * **Der Trick:** Wenn der Wert einer Kontrollvariable geändert wird (entweder durch den Benutzer in einem Widget oder durch unseren Code), werden **alle** Widgets, die an diese Variable gebunden sind, **automatisch und sofort aktualisiert**.

**Die vier wichtigsten Kontrollvariablen:**

| Klasse | Speichert | Typischer Anwendungsfall |
| :--- | :--- | :--- |
| `tk.StringVar()` | Zeichenketten (Strings) | `Entry`, `Label`, `Radiobutton` |
| `tk.IntVar()` | Ganzzahlen (Integers) | `Checkbutton` (0/1), `Radiobutton` |
| `tk.DoubleVar()` | Fließkommazahlen (Floats) | Spezielle Eingabefelder, Skalen |
| `tk.BooleanVar()` | Wahrheitswerte (`True`/`False`) | `Checkbutton` |

**Methoden der Kontrollvariablen:**

Alle diese Variablen haben zwei zentrale Methoden, die wir ständig verwenden:

| Methode | Beschreibung | Beispiel |
| :--- | :--- | :--- |
| `.set(wert)` | Setzt den Wert der Variable programmatisch. | `my_string_var.set("Neuer Text")` |
| `.get()` | Gibt den aktuellen Wert der Variable zurück. | `current_value = my_string_var.get()` |

### 4.3 Widgets an Variablen binden

Die Verbindung zwischen einem Widget und einer Kontrollvariable wird über spezielle Attribute hergestellt:

  * `textvariable`: Wird für Widgets verwendet, die Text anzeigen oder bearbeiten, wie `Label` und `Entry`.
  * `variable`: Wird für Auswahl-Widgets wie `Checkbutton` und `Radiobutton` verwendet.

-----

### 4.4 Auswahl-Widgets im Detail

Mit Kontrollvariablen im Gepäck können wir uns nun zwei neue, sehr nützliche Widgets ansehen.

#### **Das `Checkbutton`-Widget** ✅

Ein `Checkbutton` ist ein Ankreuzfeld. Es repräsentiert eine Option, die entweder **an** oder **aus** sein kann.

**Wichtige `Checkbutton`-Attribute**

| Attribut | Beschreibung | Beispiel-Werte |
| :--- | :--- | :--- |
| `master` | Das Eltern-Widget. | `window` |
| `text` | Die Beschriftung der Checkbox. | `"AGB akzeptieren"` |
| `variable` | Die Kontrollvariable, die den Zustand speichert. | `tk.BooleanVar()`, `tk.IntVar()`, `tk.StringVar()` |
| `onvalue` | Der Wert, den die `variable` annimmt, wenn die Box angekreuzt ist. | `True`, `1`, `"YES"` |
| `offvalue` | Der Wert, den die `variable` annimmt, wenn die Box nicht angekreuzt ist. | `False`, `0`, `"NO"` |
| `command` | Eine Callback-Funktion, die bei jeder Zustandsänderung aufgerufen wird. | `command=update_status` |
| `state` | Aktiviert oder deaktiviert die Checkbox. | `"normal"`, `"disabled"` |

**Isoliertes Code-Beispiel: `Checkbutton` steuert einen Button**

```python
import tkinter as tk

def toggle_button_state():
    """Überprüft den Zustand der Checkbox und (de)aktiviert den Button."""
    if terms_accepted.get(): # .get() liefert True oder False
        submit_button.config(state="normal")
    else:
        submit_button.config(state="disabled")

window = tk.Tk()
window.title("Checkbutton-Beispiel")

# Wir verwenden eine BooleanVar, die True/False speichern kann.
terms_accepted = tk.BooleanVar()

# Die Checkbox ist mit der Variable und der Callback-Funktion verbunden.
terms_check = tk.Checkbutton(
    window,
    text="Ich akzeptiere die AGB.",
    variable=terms_accepted, # Bindung an die Variable
    command=toggle_button_state
)
terms_check.pack(pady=10)

# Der Button ist anfangs deaktiviert.
submit_button = tk.Button(window, text="Absenden", state="disabled")
submit_button.pack(pady=10)

window.mainloop()
```

-----

#### **Das `Radiobutton`-Widget** 🔘

`Radiobutton`-Widgets werden immer in Gruppen verwendet. Sie erlauben dem Benutzer, **genau eine** Option aus mehreren möglichen auszuwählen.

**Das Kernkonzept:** Alle `Radiobutton`-Widgets in einer Gruppe müssen sich **dieselbe Kontrollvariable** teilen. Jeder einzelne Button hat einen einzigartigen `value`, den er in diese gemeinsame Variable schreibt, wenn er ausgewählt wird. So weiß die Variable immer, welche Option gerade aktiv ist.

**Wichtige `Radiobutton`-Attribute**

| Attribut | Beschreibung | Beispiel-Werte |
| :--- | :--- | :--- |
| `master` | Das Eltern-Widget. | `window` |
| `text` | Die Beschriftung des Radiobuttons. | `"Option A"` |
| `variable` | Die **gemeinsame** Kontrollvariable für die gesamte Gruppe. | `my_choice_var = tk.StringVar()` |
| `value` | Der **eindeutige** Wert, den dieser Button repräsentiert. | `"A"`, `"B"`, `1`, `2` |
| `command` | Eine Callback-Funktion, die beim Auswählen aufgerufen wird. | `command=show_choice` |
| `state` | Aktiviert oder deaktiviert den Button. | `"normal"`, `"disabled"` |

**Isoliertes Code-Beispiel: Eine einfache Auswahl**

```python
import tkinter as tk

def show_selected_option():
    """Zeigt die aktuell ausgewählte Option an."""
    result_label.config(text=f"Ihre Wahl: {transport_choice.get()}")

window = tk.Tk()
window.title("Radiobutton-Beispiel")

# Eine einzige StringVar für die gesamte Gruppe.
# Wir können einen Startwert setzen.
transport_choice = tk.StringVar(value="Auto")

# Label für die Frage
tk.Label(window, text="Wie reisen Sie?", font=("Arial", 14)).pack(pady=10)

# Radiobuttons erstellen. Alle teilen sich 'transport_choice'.
# Jeder hat einen einzigartigen 'value'.
radio1 = tk.Radiobutton(window, text="Auto", variable=transport_choice, value="Auto", command=show_selected_option)
radio2 = tk.Radiobutton(window, text="Zug", variable=transport_choice, value="Zug", command=show_selected_option)
radio3 = tk.Radiobutton(window, text="Fahrrad", variable=transport_choice, value="Fahrrad", command=show_selected_option)

radio1.pack(anchor="w") # anchor="w" richtet sie linksbündig aus
radio2.pack(anchor="w")
radio3.pack(anchor="w")

# Ein Label, um das Ergebnis der Auswahl anzuzeigen
result_label = tk.Label(window, font=("Arial", 12, "italic"))
result_label.pack(pady=20)

# Beim Start einmal aufrufen, um den Initialwert anzuzeigen
show_selected_option()

window.mainloop()
```

## Das Meisterstück wird verfeinert: Der BMI-Rechner mit Datenbindung

Wir bauen unseren BMI-Rechner um. Wir ersetzen die manuelle `.get()`/`.config()`-Logik durch die elegante Datenbindung und fügen eine Auswahl für das Einheitensystem hinzu.

```python
import tkinter as tk

def calculate_bmi(*args): # *args wird für die Trace-Methode benötigt
    """Liest Werte aus den Variablen, berechnet und setzt das Ergebnis."""
    try:
        # 1. Werte direkt aus den Kontrollvariablen holen
        height = height_var.get()
        weight = weight_var.get()

        # 2. Prüfen, welches Einheitensystem gewählt ist
        unit = unit_system.get()
        if unit == "imperial":
            # Umrechnung von Pfund/Zoll in kg/m
            weight = weight * 0.453592
            height = height * 0.0254

        if height <= 0: raise ZeroDivisionError

        bmi = weight / (height * height)
        # 3. Ergebnis-Variable setzen. Das Label aktualisiert sich automatisch!
        result_var.set(f"BMI: {bmi:.2f}")

    except (ValueError, ZeroDivisionError):
        result_var.set("Fehler bei der Eingabe!")

# --- Kontrollvariablen erstellen ---
height_var = tk.DoubleVar()
weight_var = tk.DoubleVar()
result_var = tk.StringVar(value="Ergebnis: -")
unit_system = tk.StringVar(value="metric") # Standardmäßig metrisch

# --- UI Erstellung ---
window = tk.Tk()
window.title("Verfeinerter BMI Rechner")

# Frame für die Eingaben
entry_frame = tk.Frame(window, padx=10, pady=10)
entry_frame.pack()

tk.Label(entry_frame, text="Größe:").grid(row=0, column=0, sticky="w")
tk.Entry(entry_frame, textvariable=height_var).grid(row=0, column=1) # an height_var gebunden
tk.Label(entry_frame, text="Gewicht:").grid(row=1, column=0, sticky="w")
tk.Entry(entry_frame, textvariable=weight_var).grid(row=1, column=1) # an weight_var gebunden

# Frame für die Einheiten-Auswahl
unit_frame = tk.Frame(window)
unit_frame.pack(pady=5)
tk.Radiobutton(unit_frame, text="Metrisch (kg, m)", variable=unit_system, value="metric").pack(side="left")
tk.Radiobutton(unit_frame, text="Imperial (lbs, in)", variable=unit_system, value="imperial").pack(side="left")

# Button und Ergebnis-Label
tk.Button(window, text="Berechnen", command=calculate_bmi, font=("Arial", 12, "bold")).pack(pady=10)
tk.Label(window, textvariable=result_var, font=("Arial", 14, "bold")).pack(pady=10) # an result_var gebunden

window.mainloop()
```

-----

---

# Übungs-Set 4: Datenbindung und Auswahl

**Ziel:** In diesen Übungen nutzen Sie Kontrollvariablen, um Ihre GUI effizienter zu machen, und setzen die neuen Auswahl-Widgets ein.

### Aufgabe 1: Der Live-Spiegel

Erstelle eine Anwendung mit einem `Entry`-Feld und einem `Label`. Verbinde beide mit derselben `StringVar`. Der Text, den du in das `Entry`-Feld tippst, soll **sofort und live** im `Label` erscheinen, ganz ohne Button.

### Aufgabe 2: Zustands-Umschalter

Erstelle ein `Checkbutton` und ein `Entry`-Feld. Wenn die Checkbox angehakt ist, soll das `Entry`-Feld editierbar sein (`"normal"`). Wenn sie nicht angehakt ist, soll das `Entry`-Feld deaktiviert sein (`"disabled"`).
*Tipp: Eine Callback-Funktion, die an das `command`-Attribut der Checkbox gebunden ist, ist hier der einfachste Weg.*

### Aufgabe 3: Einfache Umfrage

Erstelle eine Anwendung, die eine Frage stellt, z.B. "Ihre bevorzugte Programmiersprache?". Biete drei `Radiobutton`-Optionen an: "Python", "Java", "C++". Ein `Label` darunter soll immer die aktuell ausgewählte Sprache anzeigen.

### Aufgabe 4: **Schüler-Projekt: Währungsrechner aufwerten**

Rüste deinen Währungsrechner auf:

1.  Erstelle für das Eingabefeld "Betrag" und das Ergebnis-Label passende Kontrollvariablen (`DoubleVar`/`StringVar`) und binde die Widgets mit `textvariable` daran.
2.  Ersetze die `Entry`-Felder für "Von Währung" und "Nach Währung" durch eine Gruppe von `Radiobutton`-Widgets. Erstelle mindestens drei Umrechnungsoptionen (z.B. "EUR ➔ USD", "USD ➔ JPY", "GBP ➔ EUR"). Alle Radiobuttons müssen sich eine gemeinsame `StringVar` teilen.
3.  Passe deine Berechnungsfunktion an:
      * Sie soll nun den Betrag aus der `DoubleVar` lesen.
      * Sie soll prüfen, welche Option bei den `Radiobutton`s gewählt wurde (über deren `StringVar`).
      * Basierend auf der Auswahl soll der richtige (hartcodierte) Umrechnungskurs angewendet werden.
      * Das Ergebnis soll in die Ergebnis-`StringVar` geschrieben werden.

### Aufgabe 5 (Challenge): Live-Konfigurator

Erstelle eine Anwendung, um das Aussehen eines `Label`s live zu ändern.

  * Ein `Entry`-Feld, gebunden an eine `StringVar`, steuert den Text des Labels.
  * Eine Gruppe `Radiobutton`s ("Rot", "Grün", "Blau"), gebunden an eine weitere `StringVar`, steuert die Textfarbe (`fg`) des Labels.
    *Tipp: Du brauchst eine Callback-Funktion, die bei jeder Änderung aufgerufen wird. Die `.trace_add("write", callback)`-Methode einer `StringVar` ist hierfür perfekt.*

### Aufgabe 6 (Challenge): Pizza-Bestellformular 2.0

Erstelle ein Pizza-Bestellformular mit Datenbindung:

  * `Radiobutton`s für die Größe ("Klein" - 8€, "Mittel" - 10€, "Groß" - 12€).
  * `Checkbutton`s für optionale Extras ("Käse" - 1€, "Salami" - 1.50€, "Oliven" - 1€).
  * Ein `Label`, das **live** den Gesamtpreis anzeigt.
    *Tipp: Jedes Widget braucht eine eigene Kontrollvariable und eine gemeinsame Callback-Funktion, die immer den Gesamtpreis neu berechnet und in eine Ergebnis-Variable schreibt.*